## Validation plot answer sheet

Compute a plot of the position-dependent PSF size (we'll define this as the trace we computed in the last notebook)
* Make the band an input argument, and run plots for the g-band and the i-band

Hints 
* scipy.binned_statistic_2d is the function I like to use for this
* You can use the code for the data read you used last challenge
* If you were reading large amounts of data and don't want to repeat the read, just pass the data in through your input dictionary



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
sys.path.insert(0,'/global/cfs/projectdirs/lsst/groups/SRV/gcr-catalogs-test')
import GCRCatalogs
sys.path.insert(0,'/global/cfs/projectdirs/lsst/groups/SRV/descqa')
import descqa
import glob
from PIL import Image
from IPython.display import display


%matplotlib inline

In [ ]:
from descqa.base import BaseValidationTest, TestResult
import numpy as np
import healpy as hp
import os
import matplotlib.pyplot as plt
import matplotlib as mpl


from scipy.stats import binned_statistic_2d as bs2d
def trace(ixx,ixy,iyy):
    return (ixx+iyy)

    
class Tutorial3(BaseValidationTest):
    """
    validation test to plot RA/DEC
    """
    def __init__(self, **kwargs): #pylint: disable=W0231
        # get the inputs
        self.ra = kwargs.get('ra')
        self.dec = kwargs.get('dec')
        self.band = kwargs.get('band')
        self.catalog_filters = kwargs.get('catalog_filters', [])
        self.quantities =  kwargs.get('quantities', [])



    def run_on_single_catalog(self, catalog_instance, catalog_name, output_dir):


        if not catalog_instance.has_quantities([self.ra, self.dec]):
            return TestResult(skipped=True)
        filters=[]
        for i, filt in enumerate(self.catalog_filters):
            filters.append(filt)

            
        # read in the data 
        data = catalog_instance.get_quantities(self.quantities,filters=filters, return_iterator=False)
        
        data['T_PSF'] = trace(data[self.band+'_ixxPSF'],data[self.band+'_ixyPSF'],data[self.band+'_iyyPSF'])
        
        plt.title(self.band+'-band PSF size variation')
        val, xedge, yedge, bincount = bs2d(data["coord_ra"],data["coord_dec"],data['T_PSF'],statistic='mean',bins=100)
        plt.imshow(val,extent=[xedge[0],xedge[-1],yedge[0],yedge[-1]])
        plt.colorbar()
        plt.xlabel('RA (degrees)')
        plt.ylabel('dec (degrees)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'test_histogram.png'))
        plt.close()


        return TestResult(inspect_only=True, summary='made a histogram to inspect!')

In [ ]:
band = 'g'
dict_config = {}
dict_config['ra'] = 'coord_ra'
dict_config['dec'] = 'coord_dec'
dict_config['band'] = band
dict_config['quantities'] = ["coord_ra","coord_dec", band+"_ixxPSF", band+"_ixyPSF", band+"_iyyPSF",band+"_cModelFlux"]
dict_config['catalog_filters'] =  ['detect_isPrimary','refExtendedness==1']


catalog = 'lsst_object'
catalog_instance =  GCRCatalogs.load_catalog(catalog)   
output_dir_this = './test_direc'  


validation3 = Tutorial3(**dict_config)
test_result = validation3.run_on_single_catalog(catalog_instance, catalog, output_dir_this)

for image in glob.glob( output_dir_this + '/*.png'):
    print(image)
    display(Image.open(image))


# Challenges!

Okay you should have everything you need to get started looking at and validating the data. Go ahead and make your own plots! 

I have some suggestions below that the SRV group would love to get some new validation tests drafted for, but anything you or your working group wants to know about the data before it goes through an analysis pipeline is fair game! Or just test out using the DP0.2 data for your own analysis 

Some FAQs:
* There are no photometric redshifts available for this data at this part of the pipleline, see the RAIL tutorial for more details on those
* There are no metadetect shears available due to development timelines, however HSM moments are available in the catalog
* If you want direct comparison tests we can match to the truth catalog for DP0.2 (and with SSI catalogs for testing), I've provided some code below to get you a matched catalog for this tract 

## Tasks! 

* Estimate the 5-sigma limiting magnitude (using the psfFlux and definitions as described in https://dmtn-296.lsst.io/) and make a plot of the average value over RA/dec
    * Repeat for all bands (grizuy) 
* Inspect the cModel magnitudes, calib magnitudes and gaap magnitudes and colors against the truth values
    * What conclusions can you draw about the best colors to use?
    * Do these agree with the suggestions noted in the photometry flags section of https://lsstdesc.org/srv-dp02/tutorials/Rubin/tutorial7_SRV_releases.html ?
* Find a few of the brightest stars, using extendedness and magnitude cuts. Look for the RA/dec position of these and plot the galaxy density in this location. What do you see?
    * Extra credit exercise for those with DP0 delegate accounts, use the tutorials to find the tract/patch of this co-ordinate and locate and plot the deepcoadd image for this area
* Compare the gaap photometry for different apertures including the optimal aperture
    * Look through here to understand the method: https://arxiv.org/pdf/1507.00738
    * Run the srv_gaap validation code on the dp02_object_single_tract catalog (https://github.com/LSSTDESC/descqa/blob/SRV/descqa/srv_gaap.py) and see if our results agree
    * Have a look [here](https://lsstdesc.org/srv-dp02/tutorials/Rubin/tutorial7_SRV_releases.html) and the linked documentation. Do you see hints of this issue when looking at the data? How can we expand our testing to find this?


## Truth match FAQ

Some of these tasks require you to match against the truth. This requires a bit of table manipulation and requires you to know data locations and matching IDs so I'm going to put some code here so you can get the matched table and get going!

First we get the paths of a truth summary and the match to the objectTables. Now these are actually slightly different than you'll see in the [schema](https://sdm-schemas.lsst.io/dp02.html#MatchesTruth.match_chisq) because we're using the DC2 truth tables rather than the DP0.2 ones. But the input galaxies are the same so we can select on galaxies and then use this to match.  

TLDR; Run the below and use matched_table for your matched catalog. Look at truth_pandas.keys() for the values you have easy access to - more complex inputs (e.g. shapes) require a further match to the cosmodc2 catalogs. 

In [ ]:
path_truth = '/global/cfs/projectdirs/lsst/shared/DC2-prod/Run2.2i/truth/truth_merged_summary_v1-0-0/raw/truth_tract4850.parquet'
path_truth_match = '/global/cfs/projectdirs/lsst/shared/rubin/DP0.2/match_ref_truth_summary_objectTable/match_ref_truth_summary_objectTable_tract_4850_DC2_u_ctslater_matchObjectToTruth_w22_20220527T164747Z.parq'

In [ ]:
from pyarrow import dataset as ds
import pandas
import numpy as np

# let's read in the truth tables and the matching tables  - we ignore metadata here because sometimes it 
# converts an apparent ID column into the indexing and suppresses it
truth = ds.dataset(path_truth)
match = ds.dataset(path_truth_match)
truth_pandas = truth.to_table().to_pandas(ignore_metadata=True)
match_pandas = match.to_table().to_pandas(ignore_metadata=True)

In [ ]:
# we can look at the keys to confirm the schema
print(truth_pandas.keys())
print(match_pandas.keys())

In [ ]:
# now pandas doesn't like matching on int64s so we will create a string id to make it happy, 
# and filter out those non-galaxy truth objects
match_pandas['id_str'] = match_pandas['id'].astype('str')
truth_gals = truth_pandas[truth_pandas['truth_type']==1]

In [ ]:
# finally we also can't forget that we need the object catalog
catalog = 'lsst_object'
catalog_instance =  GCRCatalogs.load_catalog(catalog)   
sources = [parquet_data.path for parquet_data in catalog_instance._datasets]
dataset = ds.dataset(sources)
dataset = dataset.to_table().to_pandas(ignore_metadata=True) # otherwise it'll suppress objectID



In [ ]:
# let's do a merge - this requires objectID to be unique, but multiple object IDs can match to the same truth object
matched_table = pandas.merge(truth_gals,match_pandas,how='inner',left_on='id_string', right_on='id_str', sort=False, suffixes=('_truth','_match'),validate='one_to_many')
matched_table = pandas.merge(dataset,matched_table,how='inner',left_on='objectId', right_on='match_objectId', sort=False, suffixes=('_object',''),validate='one_to_many')
# the suffixes add a suffix to identically named quantities

Note that the truth matches aren't unique here, so you can further filter them by the chi squared value for the match if you want a better match - let's do a quick check of our match up to make sure the matches are at the same position!

In [ ]:
import matplotlib.pyplot as plt
plt.title('catalog positions')
plt.hist2d(matched_table['coord_ra'],matched_table['coord_dec'],bins=100)
plt.show()
plt.title('truth positions')
plt.hist2d(matched_table['ra'],matched_table['dec'],bins=100)
plt.show()
plt.title('positional differences')
plt.hist(matched_table['coord_ra']-matched_table['ra'],bins=100)
plt.hist(matched_table['coord_dec']-matched_table['dec'],bins=100)
plt.xlabel('degrees')
plt.show()